##Mount Azure Data Lake containers using Service Principal

In [0]:
def mount_adls(storage_account_name, container_name):
    #get secrets from key vault
    client_id = dbutils.secrets.get(scope='formula1-scope',key='formula1-clientid')
    tenant_id = dbutils.secrets.get(scope='formula1-scope',key='formula1-tenantid')
    client_secret = dbutils.secrets.get(scope='formula1-scope',key='formula1-client-secret')

    #set spark configurations
    configs = {"fs.azure.account.auth.type": "OAuth",
              "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
              "fs.azure.account.oauth2.client.id": client_id,
              "fs.azure.account.oauth2.client.secret": client_secret,
              "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"}
    
    # Unmount the mount point if it already exists
    if any(mount.mountPoint == f"/mnt/{storage_account_name}/{container_name}" for mount in dbutils.fs.mounts()):
        dbutils.fs.unmount(f"/mnt/{storage_account_name}/{container_name}")
    
    #mount storage container
    dbutils.fs.mount(
      source = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/",
      mount_point = f"/mnt/{storage_account_name}/{container_name}",
      extra_configs = configs)
    
    display(dbutils.fs.mounts())

In [0]:
#mount raw container
mount_adls('formula1racedataanalysis','raw')

/mnt/formula1racedataanalysis/raw has been unmounted.


mountPoint,source,encryptionType
/databricks-datasets,databricks-datasets,
/Volumes,UnityCatalogVolumes,
/databricks/mlflow-tracking,databricks/mlflow-tracking,
/databricks-results,databricks-results,
/databricks/mlflow-registry,databricks/mlflow-registry,
/Volume,DbfsReserved,
/mnt/formula1racedataanalysis/raw,abfss://raw@formula1racedataanalysis.dfs.core.windows.net/,
/volumes,DbfsReserved,
/,DatabricksRoot,
/volume,DbfsReserved,


In [0]:
#mount processed container
mount_adls('formula1racedataanalysis','processed')

mountPoint,source,encryptionType
/databricks-datasets,databricks-datasets,
/Volumes,UnityCatalogVolumes,
/databricks/mlflow-tracking,databricks/mlflow-tracking,
/databricks-results,databricks-results,
/databricks/mlflow-registry,databricks/mlflow-registry,
/Volume,DbfsReserved,
/mnt/formula1racedataanalysis/raw,abfss://raw@formula1racedataanalysis.dfs.core.windows.net/,
/volumes,DbfsReserved,
/mnt/formula1racedataanalysis/processed,abfss://processed@formula1racedataanalysis.dfs.core.windows.net/,
/,DatabricksRoot,


In [0]:
#mount presentation container
mount_adls('formula1racedataanalysis','presentation')

mountPoint,source,encryptionType
/databricks-datasets,databricks-datasets,
/Volumes,UnityCatalogVolumes,
/databricks/mlflow-tracking,databricks/mlflow-tracking,
/mnt/formula1racedataanalysis/presentation,abfss://presentation@formula1racedataanalysis.dfs.core.windows.net/,
/databricks-results,databricks-results,
/databricks/mlflow-registry,databricks/mlflow-registry,
/Volume,DbfsReserved,
/mnt/formula1racedataanalysis/raw,abfss://raw@formula1racedataanalysis.dfs.core.windows.net/,
/volumes,DbfsReserved,
/mnt/formula1racedataanalysis/processed,abfss://processed@formula1racedataanalysis.dfs.core.windows.net/,
